5 - É possível identificar casos inconsistentes utilizando classificação de NCM?

- CÓDIGO NCM/SH

- DATA EMISSÃO MES, DATA EMISSÃO DIA DA SEMANA, DATA EMISSÃO DIA, DATA EMISSÃO PERÍODO, VALOR DA NOTA FISCAL, VALOR UNITÁRIO, QUANTIDADE, CFOP, INDICADOR IE DESTINATÁRIO

### Importação de Bibliotecas

In [1]:
import pandas as pd

from classes import Preprocessing

2023-04-03 06:29:17.461509: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-03 06:29:17.639949: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-03 06:29:17.641795: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 06:29:20.911490: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package stopwords to /home/mayara/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mayara/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /

### Leitura de Dados

In [2]:
raw_df = pd.read_csv('data/processed_nfe/nfe_100000.csv')

In [3]:
raw_df.columns

Index(['CHAVE DE ACESSO', 'MODELO', 'SÉRIE', 'NÚMERO', 'NATUREZA DA OPERAÇÃO',
       'DATA EMISSÃO', 'EVENTO MAIS RECENTE', 'DATA/HORA EVENTO MAIS RECENTE',
       'CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE',
       'INSCRIÇÃO ESTADUAL EMITENTE', 'UF EMITENTE', 'MUNICÍPIO EMITENTE',
       'CNPJ DESTINATÁRIO', 'NOME DESTINATÁRIO', 'UF DESTINATÁRIO',
       'INDICADOR IE DESTINATÁRIO', 'DESTINO DA OPERAÇÃO', 'CONSUMIDOR FINAL',
       'PRESENÇA DO COMPRADOR', 'VALOR NOTA FISCAL', 'NÚMERO PRODUTO',
       'DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/SH',
       'NCM/SH (TIPO DE PRODUTO)', 'CFOP', 'QUANTIDADE', 'UNIDADE',
       'VALOR UNITÁRIO', 'VALOR TOTAL', 'ANO MES'],
      dtype='object')

### Pré-processamento

Tipagem

In [4]:
df = Preprocessing.define_types(raw_df)
df = Preprocessing.filter_event_authorized(df)

Definição de colunas

In [5]:
df['CAPÍTULO NCM'] = df['CÓDIGO NCM/SH'].astype(str).str[0] + df['CÓDIGO NCM/SH'].astype(str).str[1]
columns = ['DATA EMISSÃO MES', 'DATA EMISSÃO DIA SEMANA', 'DATA EMISSÃO DIA', 'DATA EMISSÃO PERIODO', 'VALOR NOTA FISCAL', 'VALOR UNITÁRIO', 'QUANTIDADE', 'CFOP', 'INDICADOR IE DESTINATÁRIO','CAPÍTULO NCM']
df = df[columns]

df.head()

/tmp/ipykernel_11333/379149.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CAPÍTULO NCM'] = df['CÓDIGO NCM/SH'].astype(str).str[0] + df['CÓDIGO NCM/SH'].astype(str).str[1]


,DATA EMISSÃO MES,DATA EMISSÃO DIA SEMANA,DATA EMISSÃO DIA,DATA EMISSÃO PERIODO,VALOR NOTA FISCAL,VALOR UNITÁRIO,QUANTIDADE,CFOP,INDICADOR IE DESTINATÁRIO,CAPÍTULO NCM
0,Março,Quarta-feira,30,Tarde,601.31,3.53,5.0,5102,NÃO CONTRIBUINTE,70
2,Julho,Sexta-feira,1,Início da tarde,38305.24,766.10,1.0,1908,CONTRIBUINTE ISENTO,90
3,Dezembro,Quarta-feira,7,Início da tarde,22285.20,20.95,504.0,1202,NÃO CONTRIBUINTE,19
4,Dezembro,Terça-feira,14,Tarde,758.90,421.30,1.0,6923,CONTRIBUINTE ISENTO,29
5,Setembro,Quinta-Feira,29,Início da tarde,2417.60,119.90,2.0,6102,NÃO CONTRIBUINTE,84


Selecionando 50 NCMs mais comuns

In [6]:
ncms_list = df['CAPÍTULO NCM'].value_counts()[0:50].index.tolist()
df = df[df['CAPÍTULO NCM'].isin(ncms_list)]

Transformando variáveis categóricas em dummies

In [7]:
# get the dummies and store it in a variable
X_orig_columns = ['DATA EMISSÃO MES', 'DATA EMISSÃO DIA SEMANA', 'DATA EMISSÃO DIA', 'DATA EMISSÃO PERIODO', 'VALOR NOTA FISCAL', 'VALOR UNITÁRIO', 'QUANTIDADE', 'CFOP', 'INDICADOR IE DESTINATÁRIO']
cat_columns = ['DATA EMISSÃO MES', 'DATA EMISSÃO DIA SEMANA', 'DATA EMISSÃO DIA', 'DATA EMISSÃO PERIODO', 'CFOP', 'INDICADOR IE DESTINATÁRIO']
dummies = pd.get_dummies(df[cat_columns])
 
# Concatenate the dummies to original dataframe
df = pd.concat([df, dummies], axis='columns')
df.drop(columns=cat_columns, inplace=True)

In [8]:
X_columns = df.columns.tolist()
X_columns.remove('CAPÍTULO NCM')

X_columns

['VALOR NOTA FISCAL',
 'VALOR UNITÁRIO',
 'QUANTIDADE',
 'DATA EMISSÃO MES_Abril',
 'DATA EMISSÃO MES_Agosto',
 'DATA EMISSÃO MES_Dezembro',
 'DATA EMISSÃO MES_Fevereiro',
 'DATA EMISSÃO MES_Janeiro',
 'DATA EMISSÃO MES_Julho',
 'DATA EMISSÃO MES_Junho',
 'DATA EMISSÃO MES_Maio',
 'DATA EMISSÃO MES_Março',
 'DATA EMISSÃO MES_Novembro',
 'DATA EMISSÃO MES_Outubro',
 'DATA EMISSÃO MES_Setembro',
 'DATA EMISSÃO DIA SEMANA_Domingo',
 'DATA EMISSÃO DIA SEMANA_Quarta-feira',
 'DATA EMISSÃO DIA SEMANA_Quinta-Feira',
 'DATA EMISSÃO DIA SEMANA_Segunda-feira',
 'DATA EMISSÃO DIA SEMANA_Sexta-feira',
 'DATA EMISSÃO DIA SEMANA_Sábado',
 'DATA EMISSÃO DIA SEMANA_Terça-feira',
 'DATA EMISSÃO PERIODO_Início da manhã',
 'DATA EMISSÃO PERIODO_Início da tarde',
 'DATA EMISSÃO PERIODO_Madrugada',
 'DATA EMISSÃO PERIODO_Manhã',
 'DATA EMISSÃO PERIODO_Noite',
 'DATA EMISSÃO PERIODO_Tarde',
 'CFOP_1101',
 'CFOP_1102',
 'CFOP_1111',
 'CFOP_1152',
 'CFOP_1201',
 'CFOP_1202',
 'CFOP_1203',
 'CFOP_1403',
 'CFOP

In [11]:
df_train, df_val, df_test = Preprocessing().split_dataset(df,X_columns,'CAPÍTULO NCM')

train: 70%
val: 10%
test: 20%


/home/mayara/Documentos/tcc/nfe-analysis/Nfe-Analysis/classes/preprocessing.py:190: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train[columns] = X['train'][columns]
/home/mayara/Documentos/tcc/nfe-analysis/Nfe-Analysis/classes/preprocessing.py:190: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train[columns] = X['train'][columns]
/home/mayara/Documentos/tcc/nfe-analysis/Nfe-Analysis/classes/preprocessing.py:190: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many

In [12]:
df_train

,VALOR NOTA FISCAL,VALOR UNITÁRIO,QUANTIDADE,DATA EMISSÃO MES_Abril,DATA EMISSÃO MES_Agosto,DATA EMISSÃO MES_Dezembro,DATA EMISSÃO MES_Fevereiro,DATA EMISSÃO MES_Janeiro,DATA EMISSÃO MES_Julho,DATA EMISSÃO MES_Junho,...,CFOP_6923,CFOP_6924,CFOP_6929,CFOP_6933,CFOP_6934,CFOP_6949,INDICADOR IE DESTINATÁRIO_CONTRIBUINTE ICMS,INDICADOR IE DESTINATÁRIO_CONTRIBUINTE ISENTO,INDICADOR IE DESTINATÁRIO_NÃO CONTRIBUINTE,CAPÍTULO NCM
33963,100.28,90.28,1.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,90
68709,520537.87,2818.17,1.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,90
14755,1560.00,390.00,4.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,85
9804,3981.00,3.12,4.0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,39
94056,3544.69,11.05,5.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46460,5611.21,161.24,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,90
89424,4562.92,2.20,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,34
68974,357.86,15.37,1.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,49
33513,6692.16,242.50,3.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,84


### Classification

##### Decision Tree

In [16]:
df_train

,VALOR NOTA FISCAL,VALOR UNITÁRIO,QUANTIDADE,DATA EMISSÃO MES_Abril,DATA EMISSÃO MES_Agosto,DATA EMISSÃO MES_Dezembro,DATA EMISSÃO MES_Fevereiro,DATA EMISSÃO MES_Janeiro,DATA EMISSÃO MES_Julho,DATA EMISSÃO MES_Junho,...,CFOP_6923,CFOP_6924,CFOP_6929,CFOP_6933,CFOP_6934,CFOP_6949,INDICADOR IE DESTINATÁRIO_CONTRIBUINTE ICMS,INDICADOR IE DESTINATÁRIO_CONTRIBUINTE ISENTO,INDICADOR IE DESTINATÁRIO_NÃO CONTRIBUINTE,CAPÍTULO NCM
33963,100.28,90.28,1.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,90
68709,520537.87,2818.17,1.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,90
14755,1560.00,390.00,4.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,85
9804,3981.00,3.12,4.0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,39
94056,3544.69,11.05,5.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46460,5611.21,161.24,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,90
89424,4562.92,2.20,2.0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,34
68974,357.86,15.37,1.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,49
33513,6692.16,242.50,3.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,84


In [14]:
from sklearn.tree import DecisionTreeClassifier
SEED=42
clf = DecisionTreeClassifier(criterion='gini', max_depth=10, max_features=0.4, splitter='best',random_state=SEED)
clf.fit(df_train[X_columns], df_train['CAPÍTULO NCM'])
df_pred = clf.predict(df_val[X_columns])

In [15]:
from sklearn.metrics import accuracy_score

print(accuracy_score(df_val['CAPÍTULO NCM'], df_pred))

0.34986625055728937


In [38]:
from sklearn.model_selection import GridSearchCV

params = {
    'criterion':  ['gini', 'entropy'],
    'max_depth':  [None, 2, 4, 6, 8, 10],
    'max_features': [None, 'sqrt', 'log2', 0.2, 0.4, 0.6, 0.8],
    'splitter': ['best', 'random']
}

clf = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid=params,
    cv=5,
    n_jobs=5,
    verbose=1,
)

clf.fit(df_val[X_columns], df_val['CAPÍTULO NCM'])
print(clf.best_params_)

Fitting 5 folds for each of 168 candidates, totalling 840 fits
{'criterion': 'gini', 'max_depth': 10, 'max_features': 0.4, 'splitter': 'best'}


#### Multilayer Perceptron